In [2]:
import os

In [3]:
%pwd

'c:\\Users\\asus\\Desktop\\Text-Summarization-Case-Study\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\asus\\Desktop\\Text-Summarization-Case-Study'

In [6]:
# Create Entity. Return type of a function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_url: str
  local_data_file: Path
  unzip_dir: Path

In [7]:
# Update Configuration Manager
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
  def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)

    create_directories([self.config.artifacts_root])


# Data Ingestion Configuration present in config.yaml
  def get_data_ingestion_config(self) -> DataIngestionConfig:
    config = self.config.data_ingestion

    create_directories([config.root_dir])

    data_ingestion_config = DataIngestionConfig(
      root_dir = config.root_dir,
      source_url = config.source_url,
      local_data_file = config.local_data_file,
      unzip_dir = config.unzip_dir
    )

    return data_ingestion_config

In [8]:
# Create Components
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
    self.config = config

# Downlading the data file from url to artifacts location
  def download_file(self):
    if not os.path.exists(self.config.local_data_file):
      filename, headers = request.urlretrieve(
        url = self.config.source_url,
        filename = self.config.local_data_file
      )
      logger.info(f"{filename} downloaded! with follwing info: \n{headers}")
    else:
      logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


# Extracting zip file(local data file) and unzipping it to unzip file
  def extract_zip_file(self):
    unzip_path = self.config.unzip_dir
    os.makedirs(unzip_path, exist_ok=True)
    with zipfile.ZipFile(self.config.local_data_file, 'r') as zipref:
      zipref.extractall(unzip_path)
      logger.info(f"Unzipped file to {unzip_path}")

In [11]:
# Create a pipeline
try:
  config = ConfigurationManager()
  data_ingestion_config = config.get_data_ingestion_config()
  data_ingestion = DataIngestion(data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
except Exception as e:
  raise e

[2024-03-26 17:17:53,572: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2024-03-26 17:17:53,572: INFO: common: yaml_file: params.yaml loaded successfully]
[2024-03-26 17:17:53,572: INFO: common: created directory at: artifacts]
[2024-03-26 17:17:53,572: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-26 17:23:06,519: INFO: 3797114757: artifacts/data_ingestion/data.zip downloaded! with follwing info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2E36:227BA9:13F66C:19D19D:6602B2DD
Accept-Ranges: bytes
Date: Tue, 26 Mar 2024 11:48:10 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100028-HYD
X-Cache